In [ ]:
import os
import subprocess

dirs = os.listdir("/home/mario/holiday")

print(dirs)

In [1]:
import os
import shutil

# Walk through the directory tree starting from "/home/mario/holiday/"
for root, dirs, files in os.walk('/home/mario/holiday/'):
    # Iterate over each directory in "dirs"
    for dir in dirs:
        # Check if the directory name is "temp"
        if dir == 'temp':
            # Create the full path of the directory to be deleted
            dir_path = os.path.join(root, dir)
            # Use shutil.rmtree to delete the directory and its contents recursively
            shutil.rmtree(dir_path)
            # Print a message to confirm the deletion
            print(f'{dir_path} has been deleted.')

/home/mario/holiday/dyr/temp has been deleted.


In [ ]:
ncpus = int(os.cpu_count()-2)
dir = 'dyr'
dockm8_cmd = f'python /home/mario/DockM8/wocondock.py --software /home/mario/DockM8/software/ --proteinfile /home/mario/holiday/{dir}/receptor_protoss_prepared.pdb --pocket reference --reffile /home/mario/holiday/{dir}/crystal_ligand_protoss.sdf --dockinglibrary /home/mario/holiday/{dir}/merged_actives_decoys.sdf --idcolumn "ID" --protonation pkasolver --docking GNINA SMINA PLANTS --metric RMSD spyRMSD espsim 3DScore bestpose bestpose_GNINA bestpose_SMINA bestpose_PLANTS --nposes 10 --exhaustiveness 8 --parallel 1 --ncpus '+str(ncpus)+' --clustering KMedoids --rescoring gnina AD4 vinardo LinF9 rfscorevs chemplp'
subprocess.call(dockm8_cmd, shell=True)

In [ ]:
for dir in dirs:
    ncpus = int(os.cpu_count()-2)
    dockm8_cmd = f'python /home/mario/DockM8/wocondock.py --software /home/mario/DockM8/software/ --proteinfile /home/mario/holiday/{dir}/receptor_protoss_prepared.pdb --pocket reference --reffile /home/mario/holiday/{dir}/crystal_ligand_protoss.sdf --dockinglibrary /home/mario/holiday/{dir}/merged_actives_decoys.sdf --idcolumn "ID" --protonation pkasolver --docking GNINA SMINA PLANTS --metric RMSD spyRMSD espsim 3DScore bestpose bestpose_GNINA bestpose_SMINA bestpose_PLANTS --nposes 10 --exhaustiveness 8 --parallel 1 --ncpus '+str(ncpus)+' --clustering KMedoids --rescoring gnina AD4 vinardo LinF9 rfscorevs chemplp'
    subprocess.call(dockm8_cmd, shell=True)